In [60]:
def split_to_halves(p):
    sum_prob_half = sum(p) / 2
    i, left_sum = 1, p[0]
    while left_sum < sum_prob_half:
        left_sum += p[i]
        i += 1
    return i

def shannon_fano_encode(p, l=0, r=None):
    if r is None: 
        r = len(p)
    if r - l == 1:
        return p[l]
    elif r - l == 2:
        return {0: p[l], 1: p[r-1]}
    
    i = l + split_to_halves(p[l:r])
    return {0: shannon_fano_encode(p, l, i),
            1: shannon_fano_encode(p, i, r)}


In [61]:
prob = [1/16, 3/16, 3/16, 9/16]
prob = sorted(prob, reverse=True)
shannon_fano_encode(prob)

{0: 0.5625, 1: {0: {0: 0.1875, 1: 0.1875}, 1: 0.0625}}

In [62]:
p1 = [1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8]
shannon_fano_encode(p1)

{0: {0: {0: 0.125, 1: 0.125}, 1: {0: 0.125, 1: 0.125}},
 1: {0: {0: 0.125, 1: 0.125}, 1: {0: 0.125, 1: 0.125}}}